In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm


In [63]:
da = pd.read_csv("nhanes_2015_2016.csv")

In [64]:
da.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


In [65]:
da.columns

Index(['SEQN', 'ALQ101', 'ALQ110', 'ALQ130', 'SMQ020', 'RIAGENDR', 'RIDAGEYR',
       'RIDRETH1', 'DMDCITZN', 'DMDEDUC2', 'DMDMARTL', 'DMDHHSIZ', 'WTINT2YR',
       'SDMVPSU', 'SDMVSTRA', 'INDFMPIR', 'BPXSY1', 'BPXDI1', 'BPXSY2',
       'BPXDI2', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXLEG', 'BMXARML', 'BMXARMC',
       'BMXWAIST', 'HIQ210'],
      dtype='object')

In [66]:
#to get the women headlines,we need to mask them out
#mask = da['RIAGENDR'] == 2
mask = (da['RIDAGEYR'] >= 35) & (da['RIDAGEYR'] <= 50)&(da['RIAGENDR'] == 2)
da = da[mask]
da.head()


,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0
34,83799,NaN,NaN,NaN,2,2,37,2,1.0,4.0,...,110.0,72.0,66.6,161.6,25.5,NaN,NaN,NaN,NaN,2.0
50,83828,1.0,NaN,2.0,2,2,39,1,2.0,3.0,...,100.0,62.0,71.3,162.0,27.2,36.8,34.6,29.1,94.6,NaN
52,83832,2.0,1.0,4.0,2,2,50,1,2.0,1.0,...,NaN,NaN,105.9,157.7,42.6,29.2,35.0,40.7,129.1,NaN
55,83837,2.0,2.0,NaN,2,2,45,1,1.0,2.0,...,114.0,68.0,77.5,148.3,35.2,30.5,34.0,34.4,107.6,2.0


In [67]:
da["DMDMARTL"] = da.DMDMARTL.replace({1: "Married", 2:"Unmarried", 3:"Unmarried", 4:"Unmarried", 5:"Unmarried", 6:"Unmarried", 77:"Unmarried", 99:"Unmarried"})
print(da["DMDMARTL"])

4       Unmarried
34        Married
50        Married
52      Unmarried
55        Married
          ...    
5658    Unmarried
5685      Married
5689      Married
5721    Unmarried
5724      Married
Name: DMDMARTL, Length: 787, dtype: object


In [68]:
da['DMDEDUC2'] = da['DMDEDUC2'].replace([1.0,2.0,3.0,4.0,5.0, 9.0], ['In_College', 'In_College', 'In_College', 'In_College', 'Com_College', 'In_College'])
print(da['DMDEDUC2'])

4        In_College
34       In_College
50       In_College
52       In_College
55       In_College
           ...     
5658     In_College
5685    Com_College
5689     In_College
5721    Com_College
5724    Com_College
Name: DMDEDUC2, Length: 787, dtype: object


In [69]:
dx = da[["DMDMARTL",  "DMDEDUC2"]]
pd.crosstab(dx.DMDMARTL, dx.DMDEDUC2)

DMDEDUC2,Com_College,In_College
DMDMARTL,,
Married,162,287
Unmarried,72,266


In [70]:
dz = dx.groupby(dx.DMDEDUC2).agg({"DMDMARTL": [lambda x: np.mean(x=="Married"), np.size]})
dz.columns = ["Proportion", "Total_c"]
dz

,Proportion,Total_c
DMDEDUC2,,
Com_College,0.692308,234
In_College,0.518987,553


In [71]:
dz = dx.groupby(dx.DMDEDUC2).agg({"DMDMARTL": [lambda x: np.mean(x=="Unmarried"), np.size]})
dz.columns = ["Proportion", "Total_c"]
dz

,Proportion,Total_c
DMDEDUC2,,
Com_College,0.307692,234
In_College,0.481013,553


In [77]:
p = dz.Proportion.Com_College
n = dz.Total_c
se_Married = np.sqrt(p * (1 - p) / n)
print(se_Married)

DMDEDUC2
Com_College    0.030172
In_College     0.019627
Name: Total_c, dtype: float64
